# Limpieza de Paramount+

## Pedir datos a la API

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq
from src.buscar_peliculas import buscar_peliculas

Pedimos los datos:

In [2]:
dfparamount_raw = buscar_peliculas('Paramount+')

<Response [200]> 2 459


Fecha de consulta en la API: 17-10-2025 1:50

Verificamos que no haya perdida de datos:

In [3]:
largo_dataframe = len(dfparamount_raw[0])
largo_segun_API = dfparamount_raw[1]
print([largo_dataframe, largo_segun_API])

[459, 459]


Los largos coinciden

In [4]:
dfparamount1 = dfparamount_raw[0]

In [5]:
dfparamount1

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,11035730,Ozzy: No Escape from Now,2025,tt35922832,1508191,movie,movie
1,1535445,Gladiator II,2024,tt9218128,558449,movie,movie
2,1437213,Top Gun: Maverick,2022,tt1745960,361743,movie,movie
3,1856741,Better Man,2024,tt14260836,799766,movie,movie
4,1583908,The Menu,2022,tt9764362,593643,movie,movie
...,...,...,...,...,...,...,...
454,1302376,Piñero,2001,tt0261066,55479,movie,movie
455,1702437,Kiss the Future,2024,tt17022868,1075845,movie,movie
456,1623462,Sabaya,2021,tt13567374,776555,movie,movie
457,2749397,Smile,2023,tt22982202,1038522,movie,movie


Guardamos estos datos crudos en un csv

In [23]:
dfparamount1.to_csv('data/raw_api_paramount.csv', index=False)

## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la pelicula en la API
- `title`: Nombre de la pelicula
- `year`: Año de estreno de la pelicula
- `imdb_id`: Identificador de la pelicula en IMDB
- `tmdb_id`: Identificador de la pelicula en TMDB
- `tmdb_type`: Tipo del titulo en TMDB
- `type`: Tipo del titulo en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo peliculas, la columna `type` y `tmdb_type` no deberian entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [7]:
dfparamount1['type'].unique()

array(['movie'], dtype=object)

In [8]:
dfparamount1['tmdb_type'].unique()

array(['movie', 'tv'], dtype=object)

En la columna `type` si tiene un solo tipo, pero al parecer `tmdb_type` no, revisemos las filas con tipo distinto

In [9]:
dfparamount1[dfparamount1['tmdb_type'] == 'tv']

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
443,3199048,The Rise of Wagner,2023,tt27480377,228220,tv,movie


Revisando el titulo 'The Rise of Wagner' en la pagina de imdb, este figura como un documental, al no estar separado en capitulos lo dejaremos como tipo `movie`. Al ya ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [10]:
dfparamount2 = dfparamount1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores nan

In [11]:
dfparamount2['id'].isna().unique()

array([False])

In [12]:
dfparamount2['title'].isna().unique()

array([False])

In [13]:
dfparamount2['year'].isna().unique()

array([False])

In [14]:
dfparamount2['imdb_id'].isna().unique()

array([False])

### Duplicados

In [15]:
dfparamount2.duplicated().sum()

np.int64(0)

### Valores unicos

#### id
Deberia ser unico para cada pelicula

In [16]:
dfparamount2['id'].value_counts()

id
11035730    1
1535445     1
1437213     1
1856741     1
1583908     1
           ..
1302376     1
1702437     1
1623462     1
2749397     1
1705096     1
Name: count, Length: 459, dtype: int64

#### title
Es posible que se repita, ya que hay peliculas con el mismo titulo

In [17]:
dfparamount2['title'].value_counts()

title
Smile                            3
Pet Sematary                     2
The Addams Family                2
Mean Girls                       2
I See You                        2
                                ..
Mission: Impossible - Fallout    1
Annihilation                     1
Novocaine                        1
Sonic the Hedgehog 3             1
Chantilly Bridge                 1
Name: count, Length: 450, dtype: int64

revisemos porque se repiten los titulos de las peliculas

In [18]:
dfparamount2[dfparamount2['title'] == 'Smile']

,id,title,year,imdb_id,tmdb_id
7,1645477,Smile,2022,tt15474916,882598
441,1698365,Smile,2022,tt15751968,1051335
457,2749397,Smile,2023,tt22982202,1038522


Son distintas peliculas con mismo nombre

#### year

In [19]:
dfparamount2['year'].value_counts()

year
2024    28
2022    26
2023    25
2002    19
2005    17
2025    15
2004    15
2007    14
2013    14
1997    14
2011    13
2021    13
2018    13
2020    13
2000    13
1999    12
2014    11
2019    11
2016    11
2015    10
2009    10
1996     9
2003     9
2012     9
2010     8
1998     8
2006     8
2001     8
2017     7
2008     6
1992     6
1995     5
1991     5
1986     5
1987     5
1981     4
1985     4
1982     4
1979     3
1994     3
1993     3
1980     3
1990     3
1989     3
1988     3
1984     2
1976     2
1974     2
1955     1
1968     1
1978     1
1983     1
1933     1
Name: count, dtype: int64

#### imdb_id

In [20]:
dfparamount2['imdb_id'].value_counts()

imdb_id
tt35922832    1
tt9218128     1
tt1745960     1
tt14260836    1
tt9764362     1
             ..
tt0261066     1
tt17022868    1
tt13567374    1
tt22982202    1
tt15038732    1
Name: count, Length: 459, dtype: int64

### Tipos de columnas

In [21]:
dfparamount2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       459 non-null    int64 
 1   title    459 non-null    object
 2   year     459 non-null    int64 
 3   imdb_id  459 non-null    object
 4   tmdb_id  459 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 18.1+ KB


Los tipos de las columnas estan bien

### Analisis de columna `year`

In [22]:
dfparamount2['year'].describe()

count     459.000000
mean     2007.932462
std        13.357440
min      1933.000000
25%      2000.000000
50%      2009.000000
75%      2020.000000
max      2025.000000
Name: year, dtype: float64

todos estan dentro de un rango logico

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [24]:
dfparamount2.to_csv('data/api_paramount.csv', index=False)